In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from io import StringIO
import geopandas as gpd
import re
from io import StringIO

In [102]:
#cargar datasets en df
df_pdvs = pd.read_csv("../data/raw/pdvs.rpt", sep=",")
df_creditos = pd.read_csv("../data/raw/creditos.rpt", sep=",")
df_ventas = pd.read_csv("../data/raw/ventas.rpt", sep=",")


/var/folders/x_/ll96cm_j5r34j0gcqbw987mw0000gn/T/ipykernel_15379/2188546676.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_creditos = pd.read_csv("../data/raw/creditos.rpt", sep=",")


In [103]:
df_creditos = df_creditos.dropna(subset=['fecha'])
df_creditos['fecha'] = df_creditos['fecha'].astype(str).str.split('.').str[0]
df_creditos['fecha'] = pd.to_datetime(df_creditos['fecha'], format='%Y%m%d')
#idmerchant tipo entero
df_creditos['idmerchant'] = df_creditos['idmerchant'].astype(int)
df_creditos["dia"] = df_creditos["fecha"].dt.dayofweek
df_creditos

,idmerchant,fecha,creditos,cantidad,descripcion
0,48731,2022-01-07,100.00,1.0,BANCO INTERNACIONAL CORRIENTE
1,90124,2023-05-23,50.00,1.0,BANCO INTERNACIONAL CORRIENTE
2,98035,2022-06-20,61.00,1.0,BANCO INTERNACIONAL CORRIENTE
3,44421,2023-04-21,10.00,1.0,BANCO INTERNACIONAL CORRIENTE
4,6286,2022-09-16,100.00,1.0,BANCO INTERNACIONAL CORRIENTE
...,...,...,...,...,...
989358,100779,2023-07-21,60.00,1.0,BANCO PRODUBANCO GRUPO PROMERICA AHORRO
989359,46448,2022-08-02,30.00,1.0,BANCO PRODUBANCO GRUPO PROMERICA AHORRO
989360,46142,2021-07-12,100.00,1.0,BANCO PRODUBANCO GRUPO PROMERICA AHORRO
989361,92688,2021-07-15,376.95,1.0,BANCO PRODUBANCO GRUPO PROMERICA AHORRO


In [104]:
df_creditos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 989363 entries, 0 to 989362
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   idmerchant   989363 non-null  int64         
 1   fecha        989363 non-null  datetime64[ns]
 2   creditos     989363 non-null  float64       
 3   cantidad     989363 non-null  float64       
 4   descripcion  989363 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 45.3+ MB


In [106]:
df_pdvs = df_pdvs.dropna(subset=['idregion'])
df_pdvs['idmerchant'] = df_pdvs['idmerchant'].astype(int)
df_pdvs

,idmerchant,idregion,ciudad,canal,actividad
0,47873,2.0,PEDERNALES,R,F
1,47179,3.0,QUITO,C,F
2,43609,3.0,RUMIÑAHUI,R,F
3,47128,2.0,GUAYAQUIL,R,F
4,47130,3.0,QUITO,R,F
...,...,...,...,...,...
79618,37339,3.0,QUITO,D,A
79619,34562,3.0,QUITO,D,S
79620,37701,3.0,QUITO,D,S
79621,34340,3.0,QUITO,D,S


In [107]:
#contar los valores de idregion de df_pdvs
df_pdvs['idregion'].value_counts()

idregion
2.0       50554
3.0       29066
4357.0        2
1567.0        1
Name: count, dtype: int64

In [108]:
#contar las ciudades de df_pdvs
df_pdvs['ciudad'].value_counts()

ciudad
GUAYAQUIL    32191
QUITO        14313
CUENCA        2258
AMBATO        2137
MACHALA       1596
             ...  
PALANDA          1
HUAMBOYA         1
TANICUCHI        1
PACCHA           1
PAQUISHA         1
Name: count, Length: 286, dtype: int64

In [109]:
df_pdvs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79623 entries, 0 to 79622
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idmerchant  79623 non-null  int64  
 1   idregion    79623 non-null  float64
 2   ciudad      79623 non-null  object 
 3   canal       77001 non-null  object 
 4   actividad   58991 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.6+ MB


In [111]:
df_ventas = df_ventas.dropna(subset=['idmerchant'])
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'], format='%Y%m%d')
#cambiar la cabecera de la columna por [venta ventas	cantidad	idmerchant	producto	fecha	categoria]
df_ventas=df_ventas.reset_index()
#unir los valores de  columna index y ventas por un . en una sola columna que se llame ventas
df_ventas['ventas'] = df_ventas['index'].astype(str) + '.' + df_ventas['ventas'].astype(str)
df_ventas["dia"] = df_ventas["fecha"].dt.dayofweek


In [68]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16149282 entries, 0 to 16149281
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ventas      float64
 1   cantidad    float64
 2   idmerchant  object 
 3   producto    object 
 4   fecha       object 
 5   categoria   object 
dtypes: float64(2), object(4)
memory usage: 739.3+ MB


In [ ]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16149282 entries, 14 to 6
Data columns (total 6 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ventas      int64         
 1   cantidad    int64         
 2   idmerchant  int64         
 3   producto    object        
 4   fecha       datetime64[ns]
 5   categoria   object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 862.5+ MB
